In [89]:
import numpy as np
from collections import defaultdict
import time
import matplotlib.pyplot as plt

In [2]:
# Race Track from Sutton and Barto Figure 5.6

#Race_Track Environment idea from https://gist.github.com/pat-coady/14978332fce195ea5c1582f49a058f18

big_course = ['WWWWWWWWWWWWWWWWWW',
              'WWWWooooooooooooo+',
              'WWWoooooooooooooo+',
              'WWWoooooooooooooo+',
              'WWooooooooooooooo+',
              'Woooooooooooooooo+',
              'Woooooooooooooooo+',
              'WooooooooooWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWWooooooWWWWWWWW',
              'WWWWooooooWWWWWWWW',
              'WWWW------WWWWWWWW']


tiny_course = ['WWWWWW',
               'Woooo+',
               'Woooo+',
               'WooWWW',
               'WooWWW',
               'WooWWW',
               'WooWWW',
               'W--WWW',]

In [3]:
ACTIONS = [[1,1],[1,0],[1,-1],[-1,-1],[0,-1],[-1,0],[0,0],[0,1],[-1,1]]

In [76]:
class RaceTrack:
    
    

    def __init__(self):

        self.Velocity_Limit = 5
        self.track = big_course
        self.eps = 1
        self.eps_decay = 0.997
        self.eps_min = 0.05
        self.Velocity_1 = 0
        self.Velocity_2 = 0
        self.nA = len(ACTIONS)
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.Starting_Row = len(self.track)-1
        self.Final_Reward = 0
        self.Move_Reward = -1
        self.N = defaultdict(lambda: np.zeros(self.nA))
        self.real_policy = defaultdict(lambda: 1)

    def reset(self):
        x,y = self.touch_wall_or_reset()
        return x,y

    def get_action_prob(self,Q_S):

        action_prob = np.ones(self.nA)*(self.eps/self.nA)
        best_action = np.argmax(Q_S)
        action_prob[best_action] = (1-self.eps) + (self.eps/self.nA) 
        return action_prob

    def touch_wall_or_reset(self):
  
        Length = len(self.track[self.Starting_Row])

        while True:
            
            num = np.random.randint(Length)
            if self.track[self.Starting_Row][num] == '-':
                x = self.Starting_Row
                y = num
                return x,y

    def step(self,coordinates,action):

        #The X-Coordinate represents the distance from the top while the Y-Coordinate represents the distance from the left
        #Counterintuitive 

        self.Velocity_1 = max(1,min(self.Velocity_1+action[0],self.Velocity_Limit))
        self.Velocity_2 = max(1,min(self.Velocity_2+action[1],self.Velocity_Limit))

        # print('Coordinates: ',coordinates)
        # print('Actions: ', action)
        # print('Velocity_1: ',self.Velocity_1)
        # print('Velocity_2: ',self.Velocity_2)

        x = coordinates[0]
        y = coordinates[1]

        done = False
        if self.track[x][y] == '+':
            done = True
            return (x,y),self.Final_Reward,done

        new_x = x - self.Velocity_1
        #print('New_X = %s,Velocity_1 = %s'%(new_x,self.Velocity_1))
        new_y = y + self.Velocity_2
        #print('New_Y = %s, Velocity_2 = %s '%(new_y,self.Velocity_2))

    

        if new_x >= (len(self.track)) or new_x < 0:
            new_x = x
            #print(new_x)

        if new_y >=(len(self.track[0])) or new_y < 0:
            new_y = y
            #print(new_y)


        next_state =  (new_x,new_y)
        reward = self.Move_Reward

        #print('NextState[0]: %s, NextState[1]: %s'%(next_state[0],next_state[1]))
        if self.track[next_state[0]][next_state[1]] == 'W':
            
          #Moving Back to starting line randomly
          #print('Randomly Moving Back To Finish Line: ',next_state)
            x,y = self.touch_wall_or_reset()
            next_state = (x,y)

        return next_state,reward,done

    def get_episode(self):
        
  
        trajectory = []

        state = self.reset()

        while True:
            

            action_prob =  self.get_action_prob(self.Q[state])
            action_index = np.random.choice(np.arange(self.nA),p=action_prob)
            action = ACTIONS[action_index]
            next_state,reward,done = self.step(state,action)
            trajectory.append((state,action,reward))
            state = next_state
      
            #time.sleep(1)   
            if done:
                break

        return trajectory 

    def Update_Q(self):
        

        tra = self.get_episode()

        states,actions,rewards = zip(*tra)

        for i,state in enumerate(states):

            for j,x in enumerate(ACTIONS):

                if x == actions[i]:
                    action = j

            self.N[state][action] +=1 
            alpha = 1/self.N[state][action]
            self.Q[state][action] += alpha*(sum(rewards[i:]) - self.Q[state][action]) 

        return self.Q

    def best_policy(self):
        
        return dict((state,np.argmax(_action_))for state,_action_ in self.Q.items())

    def best_value(self):
        
        return dict((state,np.max(action))for state,action in self.Q.items())

    def Monte_Carlo_On_Policy(self,iters):

        for i in range(iters):

            self.eps = max(self.eps_min,self.eps*self.eps_decay)

            self.Q = self.Update_Q()

        policy = self.best_policy()
        self.real_policy = self.Real_Actions(policy)

        return self.Q,self.real_policy
    
    def Real_Actions(self,policy):
        states_ava = []
        for i in policy:
            states_ava.append(i)
            
        actions_ava = []
        for state in states_ava:
            action = policy.get(state)
            real_a = ACTIONS[action]
            self.real_policy[state] = real_a 
            
        return self.real_policy

In [77]:
env = RaceTrack()

In [85]:
Q,policy = env.Monte_Carlo_On_Policy(iters=10000)

In [86]:
value = env.best_value()
policy

defaultdict(<function __main__.RaceTrack.__init__.<locals>.<lambda>()>,
            {(32, 4): [0, -1],
             (31, 5): [-1, 0],
             (29, 6): [1, 0],
             (27, 7): [1, 1],
             (25, 9): [1, 1],
             (32, 9): [-1, 1],
             (32, 6): [1, 1],
             (23, 8): [0, -1],
             (18, 9): [-1, 0],
             (28, 5): [0, -1],
             (23, 6): [0, 0],
             (18, 7): [-1, -1],
             (13, 9): [0, 0],
             (32, 8): [1, -1],
             (27, 9): [-1, 1],
             (32, 5): [0, 0],
             (28, 6): [-1, 0],
             (25, 7): [-1, -1],
             (22, 8): [-1, -1],
             (20, 9): [-1, 1],
             (32, 7): [0, 0],
             (31, 8): [-1, 1],
             (30, 8): [0, 1],
             (30, 5): [0, 1],
             (22, 9): [0, -1],
             (28, 8): [-1, -1],
             (29, 9): [0, 1],
             (29, 8): [-1, -1],
             (28, 9): [-1, -1],
             (30, 9): [1, 1],
    

In [87]:
Q

defaultdict(<function __main__.RaceTrack.__init__.<locals>.<lambda>()>,
            {(32,
              4): array([-1088.58467545, -1094.01749271, -1064.62251972, -1085.42358604,
                     -251.29642117, -1090.58887317, -1129.96710208, -1093.02392344,
                    -1103.95852098]),
             (31,
              5): array([-1467.44578313, -1126.22222222, -1345.71717172, -1059.53571429,
                    -1217.1       ,  -274.32613054, -1184.96590909,  -942.90243902,
                    -1090.48863636]),
             (29,
              6): array([ -882.53918495,  -222.0551993 ,  -936.70032573, -1059.27402135,
                    -1129.97394137, -1000.88474576, -1188.74025974, -1007.9347079 ,
                    -1025.71864407]),
             (27,
              7): array([ -247.67807239, -1110.42083333,  -976.16765286, -1080.8452381 ,
                     -988.55289421, -1227.50632911, -1035.75471698, -1217.55357143,
                    -1051.5847619 ]),
            

In [88]:
value

{(32, 4): -251.29642116773823,
 (31, 5): -274.32613054132383,
 (29, 6): -222.05519930231566,
 (27, 7): -247.67807238770155,
 (25, 9): -276.6978889757626,
 (32, 9): -264.3123753676341,
 (32, 6): -263.6317024889348,
 (23, 8): -476.56907037358854,
 (18, 9): -247.9078054298651,
 (28, 5): -100.39325312573702,
 (23, 6): -313.8898678414095,
 (18, 7): -208.21500000000006,
 (13, 9): -389.63063063063066,
 (32, 8): -251.89298638635583,
 (27, 9): -250.72075971146862,
 (32, 5): -261.03719066719407,
 (28, 6): -266.1917437041758,
 (25, 7): -267.82634730539047,
 (22, 8): -234.47877083476166,
 (20, 9): -201.45262793914296,
 (32, 7): -258.6219095459023,
 (31, 8): -284.6929276155115,
 (30, 8): -276.7060957910024,
 (30, 5): -275.0512649800279,
 (22, 9): -389.1413586413579,
 (28, 8): -249.89869775570693,
 (29, 9): -302.3171417415062,
 (29, 8): -262.66177795575265,
 (28, 9): -267.5945141771274,
 (30, 9): -294.88791732909465,
 (31, 9): -357.2936111111118,
 (29, 7): -279.31734157315725,
 (27, 8): -261.3224317

In [75]:
tiny_course[5][1]

'o'